# Probability Distributions and Random Values

In [62]:
from scipy import stats
import numpy as np

## Common Reference points

### PDF Value at mean of univariate gaussian

In [63]:
stats.norm.pdf(0, loc=0, scale=1)

0.3989422804014327

### Log PDF Value at mean of univariate gaussian

In [64]:
stats.norm.logpdf(0, loc=0, scale=1)

-0.9189385332046727

## Probability Graphs
Probability can be forwarded through graphs. In the case below our model contains two distributions. Z which estimates the location of mu, and x which has one observation of 5


Below are examples in a couple APIS

$$
z \~ norm(0,5)  \\
x \~ norm(z, 1) \\
X_{observed}
$$

### Probability of Z given value of 2.5 

In [65]:
z_random_var = 2.5
z_dist = stats.norm(0,5)
z_dist

In [66]:
z_2_5_prob = z_dist.pdf(z_random_var)
z_2_5_prob

0.0704130653528599

In [67]:
z_2_5_log_prob = np.log(z_2_5_prob)
z_2_5_log_prob

-2.653376445638773

### Probability of X given observed value of 5

In [68]:
x_dist = stats.norm(loc=z_random_var, scale=1)
x_prob = x_dist.pdf(5)
x_prob

0.01752830049356854

In [69]:
x_log_prob = np.log(x_prob)
x_log_prob

-4.043938533204672

### Total Probabilty of model

In [70]:
z_2_5_log_prob + x_log_prob

-6.697314978843446

## Pymc3 Model
Pymc3 combines the notion of random variable and distribution into one line. In the model context each distribution is initialized

In [71]:
import pymc3 as pm
with pm.Model() as model:
    z = pm.Normal('z', mu=0, sd=5)
    x = pm.Normal('x', z, sd=1, observed=5)
    
print(x.logp({"z":2.5}))
model.logp({"z":2.5})

-4.043938533204672


array(-6.69731498)

Alternatively we could be more explicit about defining distributions and getting probabiities from each. By default we get theno tensors which is why we ned to call eval

In [72]:
z_log_prob = pm.Normal.dist(mu=0, sd=5).logp(z_random_var)
x_log_prob = pm.Normal.dist(z_random_var, sd=1).logp(5)
print((x_log_prob + z_log_prob).eval())

-6.697314978843446


## Tensorflow Probability
Tensorflow probability is similar. A computation graph is created and at the end eval is called to generate results.

In [73]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd

z_dist = tfd.Normal(loc=5, scale=5)
# Not needed but can serve as a standin for the Z var
z = z_dist.sample()
x = tfd.Normal(loc=z, scale=1.).log_prob(5.)
model_logp = z_dist.log_prob(2.5) + x

sess = tf.Session()
print(sess.run(x, feed_dict={z: 2.5}))
print(sess.run(model_logp, feed_dict={z: 2.5}))

-4.0439386
-6.697315
